In [1]:
'''

CINEMAC. 

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.

project for the course of Network Analysis - 2022/2023
Realized by.
	Andrea D'Arpa
	Maddalena Ghiotto		
	Chloe Papadopoulou

'''
import os
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
from ast import literal_eval
from listReader import *
%pip install unidecode
from unidecode import unidecode

# %pip install --upgrade pip
# %pip install difflib
# # import difflib




Note: you may need to restart the kernel to use updated packages.


In [2]:

# creating the tables with pandas to interact with our datasets
# getting the current working directory
cwd = os.getcwd()

name = cwd+'/datasets/name.basics.tsv'
name_table = pd.read_csv(name , sep='\t', header=0)

title = cwd+'/datasets/title.principals.tsv'
title_table = pd.read_csv(title , sep='\t', header=0)

#import titles ds to match the movie-links titles, 
title_name = cwd+'/datasets/title.basics.tsv'
title_name_table= pd.read_csv(title_name, sep='\t', header=0, low_memory=False)

                              
# # dtype={'tconst':'int64',	'titleType':'str',	'primaryTitle':'str',	'originalTitle':'str',	'isAdult':'bool',	'startYear':'int64',	'endYear':'int64',	'runtimeMinutes':'int64',	'genres':'str'})


countries = cwd+'/datasets/countries.list.gz'
countries_table = get_movie_countries(countries)
# countries_table.to_csv("countries_table.csv")

movies = cwd+'/datasets/movie-links.list.gz'
movies_table = get_movie_links(movies)
# movies_table.to_csv("movies_table.csv")

'''
testing the tables
'''
# print(movies_table.head(5))
# print(countries_table.head(5))
# print(title_table.head(5))
# print(name_table.head(5))


#merge to include countries:

movies_cit_countries = pd.merge(movies_table, countries_table, on='movie', how='left')

movies_cit_countries=pd.merge(movies_cit_countries, countries_table, left_on="cites", right_on="movie",how="left")

movies_cit_countries.rename(columns={"movie_x":"movie","movie_y":"cited_movies","country_x":"movie_country","country_y":"cites_country"},inplace=True)
movies_cit_countries=movies_cit_countries.reindex(columns= ['movie', 'movie_country', 'cites', 'movie_y','cites_country'])
movies_cit_countries.drop(columns=['movie_y'], inplace=True)
movies_cit_countries.head(100)

,movie,movie_country,cites,cites_country
0,#DevinAuditions (2016),Canada,Alien (1979),UK
1,#DevinAuditions (2016),Canada,Alien (1979),USA
2,#DevinAuditions (2016),Canada,Die Hard (1988),USA
3,#DevinAuditions (2016),Canada,Ferris Bueller's Day Off (1986),USA
4,#DevinAuditions (2016),Canada,Fight Club (1999),USA
...,...,...,...,...
95,'Joe Strummer': The Man (2007),USA,Joe Strummer: The Future Is Unwritten (2007),UK
96,'Kurenai no kenjû' yo eien ni (2000),Japan,Kurenai no kenju (1961),Japan
97,'Men Olsenbanden var ikke død!' (1984),Norway,Olsen-banden møter kongen og knekten (1974),Norway
98,'Men Olsenbanden var ikke død!' (1984),Norway,Olsen-banden og Dynamitt-Harry (1970),Norway


In [3]:
movies_cit_countries['movie_country'].unique()
movies_cit_countries['cites_country'].unique()

array(['UK', 'USA', 'Germany', 'Hong Kong', 'New Zealand', 'Australia',
       'Netherlands', 'India', 'West Germany', 'Finland', 'Poland',
       'France', 'Italy', 'Philippines', 'Mexico', 'Ireland', 'Japan',
       'Norway', 'Greece', 'Jamaica', 'Belgium', 'Spain', 'Soviet Union',
       'Canada', 'South Korea', 'Argentina', 'Bolivia', 'Sweden',
       'Bosnia and Herzegovina', 'Austria', 'Turkey', 'Vietnam',
       'East Germany', 'Taiwan', 'Czechoslovakia', 'Yugoslavia',
       'Czech Republic', 'Bahamas', 'Iran', 'Ukraine', nan, 'Singapore',
       'Thailand', 'Brazil', 'Switzerland', 'China', 'Denmark', 'Romania',
       'South Africa', 'Israel', 'Cambodia', 'Indonesia', 'Panama',
       'Portugal', 'Pakistan', 'Chile', 'Peru', 'Estonia', 'Hungary',
       'Russia', 'Ghana', 'Burkina Faso', 'Malaysia', 'Luxembourg',
       'United Arab Emirates', 'Morocco', 'Monaco', 'Mauritius', 'Serbia',
       'Albania', 'Senegal', 'Latvia', 'Bulgaria',
       'Federal Republic of Yugoslavia'

In [4]:
#retrieve information for movies collaborators

#split the movie column in the first dataframe to separate title from year

# replace /n for nan values
title_name_table = title_name_table.replace("\\N", np.nan)



# extract year: pattern of four digits between parentheses at the end of the string
pattern = r'^(.*) \((\d{4})\)$'

movies_cit_countries[['title', 'year']] = movies_cit_countries['movie'].str.extract(pattern)
movies_cit_countries[['primaryTitle_citation', 'year_citation']] = movies_cit_countries['cites'].str.extract(pattern)

movies_cit_countries.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 332925 entries, 0 to 332924
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   movie                  332925 non-null  object
 1   movie_country          332634 non-null  object
 2   cites                  332925 non-null  object
 3   cites_country          332867 non-null  object
 4   title                  332925 non-null  object
 5   year                   332925 non-null  object
 6   primaryTitle_citation  332925 non-null  object
 7   year_citation          332925 non-null  object
dtypes: object(8)
memory usage: 22.9+ MB


In [5]:
#******TROUBLESHOOTING missing values on merge ********

#encoding is same in both, utf-8 - check
#replaced /n for nan values before
#check that year format is correct everywhere
# 
non_valid_year = list([y for y in movies_cit_countries['year'].unique() if not (str(y).isdigit() and len(str(y)) == 4)])
non_valid_year
error_rows = movies_cit_countries[movies_cit_countries['year'].isin(non_valid_year)]
error_rows

non_valid_year_x = list([y for y in movies_cit_countries['year_citation'].unique() if not (str(y).isdigit() and len(str(y)) == 4)])
non_valid_year_x
error_rows_x = movies_cit_countries[movies_cit_countries['year_citation'].isin(non_valid_year_x)]
error_rows.empty and error_rows_x.empty



True

In [6]:

#remove useless whitespaces
movies_cit_countries['title'] = movies_cit_countries['title'].str.strip()
movies_cit_countries['primaryTitle_citation'] = movies_cit_countries['primaryTitle_citation'].str.strip()

title_name_table['primaryTitle'] = title_name_table['primaryTitle'].str.strip()
title_name_table['originalTitle'] = title_name_table['originalTitle'].str.strip()

#take out tv series and videogames/videos

title_name_table = title_name_table.loc[title_name_table["titleType"] != 'tvEpisode' ]
title_name_table = title_name_table.loc[title_name_table["titleType"] != 'tvSeries' ]
title_name_table = title_name_table.loc[title_name_table["titleType"] != 'videoGame' ]
title_name_table = title_name_table.loc[title_name_table["titleType"] != 'video' ]
title_name_table = title_name_table.loc[title_name_table["titleType"] != 'tvMiniSeries' ]
title_name_table = title_name_table.loc[title_name_table["titleType"] != 'tvShort' ]
title_name_table = title_name_table.loc[title_name_table["titleType"] != 'tvSpecial' ]



title_name_table.drop(columns=['endYear','runtimeMinutes'], inplace=True)


In [7]:
movies_cit_countries = movies_cit_countries.replace(['Zui jia pai dang 3: Nv huang mi ling'] ,'Zui jia pai dang 3: Nu huang mi ling')


In [8]:

#remove special characters etc

def normalize_string(s):
    s = s.replace('&', 'and')
    s = re.sub(r'[^\w\s]', '', s).lower()  # Keep alphanumeric and whitespace characters
    s = re.sub(r'\s+', '', s)  # Remove whitespace characters
    s = unidecode(s)  # Remove accents and diacritics
    return s


title_name_table['originalTitle_normalized'] = title_name_table['originalTitle'].apply(normalize_string)
movies_cit_countries['title_citation_normalized'] = movies_cit_countries['primaryTitle_citation'].apply(normalize_string)
title_name_table['primaryTitle_normalized'] = title_name_table['primaryTitle'].apply(normalize_string)


movies_cit_countries 

#make sure formats are the same
#assign best datatypes

movies_cit_countries=movies_cit_countries.convert_dtypes()
title_name_table=title_name_table.convert_dtypes()

#they all turn out as str- edit years
#remove nan values

movies_cit_countries['year'] =  movies_cit_countries['year'].fillna('0')
movies_cit_countries['year_citation'] =  movies_cit_countries['year_citation'].fillna('0')
title_name_table['startYear'] =  title_name_table['startYear'].fillna('0')


#assign types
movies_cit_countries= movies_cit_countries.astype({'year': 'int32', 'year_citation':'int32'})

title_name_table= title_name_table.astype({'startYear': 'int32'})



In [9]:
movies_cit_countries.info()
title_name_table.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 332925 entries, 0 to 332924
Data columns (total 9 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   movie                      332925 non-null  string
 1   movie_country              332634 non-null  string
 2   cites                      332925 non-null  string
 3   cites_country              332867 non-null  string
 4   title                      332925 non-null  string
 5   year                       332925 non-null  int32 
 6   primaryTitle_citation      332925 non-null  string
 7   year_citation              332925 non-null  int32 
 8   title_citation_normalized  332925 non-null  string
dtypes: int32(2), string(7)
memory usage: 22.9 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1692874 entries, 0 to 9640999
Data columns (total 9 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   tconst                    strin

In [10]:
#first merge on title 
final_df = pd.merge(movies_cit_countries, title_name_table,
             left_on=['title_citation_normalized'],
             right_on=['originalTitle_normalized'],how='outer', indicator=True)
                              

final_df.info()                     

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2798154 entries, 0 to 2798153
Data columns (total 19 columns):
 #   Column                     Dtype   
---  ------                     -----   
 0   movie                      string  
 1   movie_country              string  
 2   cites                      string  
 3   cites_country              string  
 4   title                      string  
 5   year                       float64 
 6   primaryTitle_citation      string  
 7   year_citation              float64 
 8   title_citation_normalized  string  
 9   tconst                     string  
 10  titleType                  string  
 11  primaryTitle               string  
 12  originalTitle              string  
 13  isAdult                    string  
 14  startYear                  float64 
 15  genres                     string  
 16  originalTitle_normalized   string  
 17  primaryTitle_normalized    string  
 18  _merge                     category
dtypes: category(1), float

In [11]:

#identify which are the movies that share the same year and filter
matches = final_df[final_df['_merge'] == "both"].copy()

#include 2 year differences

matches['year_diff'] = matches['year_citation'] - matches['startYear']
matches = matches.loc[matches['year_diff'].between(-3, 3)]

#now we have the further matches and we can append them
matches
              

,movie,movie_country,cites,cites_country,title,year,primaryTitle_citation,year_citation,title_citation_normalized,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,genres,originalTitle_normalized,primaryTitle_normalized,_merge,year_diff
0,#DevinAuditions (2016),Canada,Alien (1979),UK,#DevinAuditions,2016.0,Alien,1979.0,alien,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",alien,alien,both,0.0
10,#DevinAuditions (2016),Canada,Alien (1979),USA,#DevinAuditions,2016.0,Alien,1979.0,alien,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",alien,alien,both,0.0
20,.com for Murder (2002),USA,Alien (1979),UK,.com for Murder,2002.0,Alien,1979.0,alien,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",alien,alien,both,0.0
30,.com for Murder (2002),USA,Alien (1979),USA,.com for Murder,2002.0,Alien,1979.0,alien,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",alien,alien,both,0.0
40,30 Minutes or Less (2011),Germany,Alien (1979),UK,30 Minutes or Less,2011.0,Alien,1979.0,alien,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",alien,alien,both,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1224183,è solo questione di punti di vista (2012),Italy,Letto a tre piazze (1960),Italy,è solo questione di punti di vista,2012.0,Letto a tre piazze,1960.0,lettoatrepiazze,tt0054023,movie,Letto a tre piazze,Letto a tre piazze,0,1960.0,Comedy,lettoatrepiazze,lettoatrepiazze,both,0.0
1224184,è solo questione di tempo (2013),Italy,Il successo (1963),Italy,è solo questione di tempo,2013.0,Il successo,1963.0,ilsuccesso,tt0057540,movie,Il successo,Il successo,0,1963.0,Comedy,ilsuccesso,ilsuccesso,both,0.0
1224185,è solo questione di tempo (2013),Italy,Il successo (1963),Italy,è solo questione di tempo,2013.0,Il successo,1963.0,ilsuccesso,tt1248945,tvMovie,Il successo,Il successo,0,1963.0,<NA>,ilsuccesso,ilsuccesso,both,0.0
1224187,è solo questione di tempo (2013),Italy,Il successo (1963),France,è solo questione di tempo,2013.0,Il successo,1963.0,ilsuccesso,tt0057540,movie,Il successo,Il successo,0,1963.0,Comedy,ilsuccesso,ilsuccesso,both,0.0


In [12]:
#compare missing values
movies_cit_countries['movie'].unique()


<StringArray>
[                                     '#DevinAuditions (2016)',
                                        '#FromJennifer (2017)',
                                                 '#Rip (2013)',
                                         '#TubeClash02 (2016)',
 '#chicagoGirl: The Social Network Takes on a Dictator (2013)',
                                        '#twitterkills (2014)',
                                                '#will (2016)',
                                            '$10 Raise (1935)',
                               '$10,000 Under a Pillow (1921)',
                                             '$5 a Day (2008)',
 ...
                                'Úristen, itt lönek... (2005)',
                                       'Üksindusfilm 2 (2016)',
                                           'Üvegtigris (2001)',
                                        'Üvegtigris 2. (2006)',
                                        'Üvegtigris 3. (2010)',
                     

In [13]:
matches['primaryTitle_citation'].unique()

<StringArray>
[                           'Alien',                         'Die Hard',
         "Ferris Bueller's Day Off",                       'Fight Club',
                       'Magic Mike',                    'The Gold Rush',
                      'The Shining',                       '2 Jennifer',
                      'To Jennifer',                     'Love, Simple',
 ...
                'Með allt á hreinu',              'Plecarea Vlasinilor',
 'Ôgon no taka - Zempen: Makyô-hen',                   'Taii no musume',
                     'Üksindusfilm',                       'Üvegtigris',
                    'Üvegtigris 2.',                        'Smáfuglar',
               'Letto a tre piazze',                      'Il successo']
Length: 40790, dtype: string

In [14]:

imdb_unmatched = final_df[final_df['_merge'] == "right_only"].copy()
links_unmatched = final_df[final_df['_merge'] == "left_only"].copy()

imdb_unmatched
matches_1 = pd.merge(links_unmatched, imdb_unmatched,
             left_on=['title_citation_normalized'],
             right_on=['primaryTitle_normalized'],how='outer', indicator="match_primaryTitle")

matches_1


,movie_x,movie_country_x,cites_x,cites_country_x,title_x,year_x,primaryTitle_citation_x,year_citation_x,title_citation_normalized_x,tconst_x,...,titleType_y,primaryTitle_y,originalTitle_y,isAdult_y,startYear_y,genres_y,originalTitle_normalized_y,primaryTitle_normalized_y,_merge_y,match_primaryTitle
0,(T)Raumschiff Surprise - Periode 1 (2004),Germany,Le cinquième élément (1997),France,(T)Raumschiff Surprise - Periode 1,2004.0,Le cinquième élément,1997.0,lecinquiemeelement,<NA>,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
1,24x36: A Movie About Movie Posters (2016),Canada,Le cinquième élément (1997),France,24x36: A Movie About Movie Posters,2016.0,Le cinquième élément,1997.0,lecinquiemeelement,<NA>,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
2,24x36: A Movie About Movie Posters (2016),USA,Le cinquième élément (1997),France,24x36: A Movie About Movie Posters,2016.0,Le cinquième élément,1997.0,lecinquiemeelement,<NA>,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
3,Astro Boy (2009),Hong Kong,Le cinquième élément (1997),France,Astro Boy,2009.0,Le cinquième élément,1997.0,lecinquiemeelement,<NA>,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
4,Astro Boy (2009),USA,Le cinquième élément (1997),France,Astro Boy,2009.0,Le cinquième élément,1997.0,lecinquiemeelement,<NA>,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1577509,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,...,movie,Dankyavar Danka,Dankyavar Danka,0,2013.0,Comedy,dankyavardanka,dankyavardanka,right_only,right_only
1577510,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,...,short,Hay Que Ser Paciente,Hay Que Ser Paciente,0,2015.0,"Documentary,Short",hayqueserpaciente,hayqueserpaciente,right_only,right_only
1577511,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,...,movie,6 Gunn,6 Gunn,0,2017.0,<NA>,6gunn,6gunn,right_only,right_only
1577512,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,...,movie,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,0,2013.0,Documentary,chicoalbuquerquerevelacoes,chicoalbuquerquerevelacoes,right_only,right_only


In [15]:
matches_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1577514 entries, 0 to 1577513
Data columns (total 39 columns):
 #   Column                       Non-Null Count    Dtype   
---  ------                       --------------    -----   
 0   movie_x                      3588 non-null     string  
 1   movie_country_x              3581 non-null     string  
 2   cites_x                      3588 non-null     string  
 3   cites_country_x              3587 non-null     string  
 4   title_x                      3588 non-null     string  
 5   year_x                       3588 non-null     float64 
 6   primaryTitle_citation_x      3588 non-null     string  
 7   year_citation_x              3588 non-null     float64 
 8   title_citation_normalized_x  3588 non-null     string  
 9   tconst_x                     0 non-null        string  
 10  titleType_x                  0 non-null        string  
 11  primaryTitle_x               0 non-null        string  
 12  originalTitle_x             

In [16]:
matches_primaryTitle = matches_1[matches_1['match_primaryTitle'] == "both"].copy()
matches_primaryTitle.values[0]
matches_primaryTitle['year_diff_prim'] = matches_primaryTitle['year_citation_x'] - matches_primaryTitle['startYear_y']
matches_primaryTitle = matches_primaryTitle.loc[matches_primaryTitle['year_diff_prim'].between(-3, 3)]
matches_primaryTitle

,movie_x,movie_country_x,cites_x,cites_country_x,title_x,year_x,primaryTitle_citation_x,year_citation_x,title_citation_normalized_x,tconst_x,...,primaryTitle_y,originalTitle_y,isAdult_y,startYear_y,genres_y,originalTitle_normalized_y,primaryTitle_normalized_y,_merge_y,match_primaryTitle,year_diff_prim
48,(T)Raumschiff Surprise - Periode 1 (2004),Germany,Star Wars: Episode V - The Empire Strikes Back...,USA,(T)Raumschiff Surprise - Periode 1,2004.0,Star Wars: Episode V - The Empire Strikes Back,1980.0,starwarsepisodevtheempirestrikesback,<NA>,...,Star Wars: Episode V - The Empire Strikes Back,The Empire Strikes Back,0,1980.0,"Action,Adventure,Fantasy",theempirestrikesback,starwarsepisodevtheempirestrikesback,right_only,both,0.0
49,17 Again (2009),USA,Star Wars: Episode V - The Empire Strikes Back...,USA,17 Again,2009.0,Star Wars: Episode V - The Empire Strikes Back,1980.0,starwarsepisodevtheempirestrikesback,<NA>,...,Star Wars: Episode V - The Empire Strikes Back,The Empire Strikes Back,0,1980.0,"Action,Adventure,Fantasy",theempirestrikesback,starwarsepisodevtheempirestrikesback,right_only,both,0.0
50,2010: The Odyssey Continues (1984),USA,Star Wars: Episode V - The Empire Strikes Back...,USA,2010: The Odyssey Continues,1984.0,Star Wars: Episode V - The Empire Strikes Back,1980.0,starwarsepisodevtheempirestrikesback,<NA>,...,Star Wars: Episode V - The Empire Strikes Back,The Empire Strikes Back,0,1980.0,"Action,Adventure,Fantasy",theempirestrikesback,starwarsepisodevtheempirestrikesback,right_only,both,0.0
51,24x36: A Movie About Movie Posters (2016),Canada,Star Wars: Episode V - The Empire Strikes Back...,USA,24x36: A Movie About Movie Posters,2016.0,Star Wars: Episode V - The Empire Strikes Back,1980.0,starwarsepisodevtheempirestrikesback,<NA>,...,Star Wars: Episode V - The Empire Strikes Back,The Empire Strikes Back,0,1980.0,"Action,Adventure,Fantasy",theempirestrikesback,starwarsepisodevtheempirestrikesback,right_only,both,0.0
52,24x36: A Movie About Movie Posters (2016),USA,Star Wars: Episode V - The Empire Strikes Back...,USA,24x36: A Movie About Movie Posters,2016.0,Star Wars: Episode V - The Empire Strikes Back,1980.0,starwarsepisodevtheempirestrikesback,<NA>,...,Star Wars: Episode V - The Empire Strikes Back,The Empire Strikes Back,0,1980.0,"Action,Adventure,Fantasy",theempirestrikesback,starwarsepisodevtheempirestrikesback,right_only,both,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3290,Yin yang lu shi ba zhi Gui shang shen (2003),Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi ba zhi Gui shang shen,2003.0,Troublesome Night 11,2001.0,troublesomenight11,<NA>,...,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",yamyeunglo11liugwailomin,troublesomenight11,right_only,both,0.0
3291,Yin yang lu shi jiu zhi Wo dui yan jian dao ye...,Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi jiu zhi Wo dui yan jian dao ye,2003.0,Troublesome Night 11,2001.0,troublesomenight11,<NA>,...,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",yamyeunglo11liugwailomin,troublesomenight11,right_only,both,0.0
3292,Yin yang lu shi liu zhi hui dao wu xia shi dai...,Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi liu zhi hui dao wu xia shi dai,2002.0,Troublesome Night 11,2001.0,troublesomenight11,<NA>,...,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",yamyeunglo11liugwailomin,troublesomenight11,right_only,both,0.0
3293,Yin yang lu shi qi zhi jian fang you gui (2002),Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi qi zhi jian fang you gui,2002.0,Troublesome Night 11,2001.0,troublesomenight11,<NA>,...,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",yamyeunglo11liugwailomin,troublesomenight11,right_only,both,0.0


In [17]:
links_missing_completely = matches_1[matches_1['match_primaryTitle'] == "left_only"].copy()#show non matched data
#to check:
links_missing_completely['primaryTitle_citation_x'].unique()

<StringArray>
[                         'Le cinquième élément',
                      'The 12 Dogs of Christmas',
      'Ging chaat goo si 4: Ji gaan daan yam mo',
                '2012 Gold Rush Expedition Race',
                              'Ah fei zing zyun',
                              'Faa yeung nin wa',
      'Non si deve profanare il sonno dei morti',
                             'Jeg, en kvinda II',
              'Yu pu tuan er zhi Yu nu xin jing',
                 'Yu pu tuan: Tou qing bao jian',
 ...
      'Yin yang lu shi qi zhi jian fang you gui',
         'Yin yang lu shi ba zhi Gui shang shen',
                                 'The Goldbergs',
                                       'Yu feng',
                                 'Detstvo Bambi',
 'Yôjû kyôshitsu gaiden 2: Eros no megami kôrin',
                             'Cossacks in Exile',
                                        'Ji sor',
                    'Zhi zun ji zhuang yuan cai',
                               

In [18]:
#remove columns with all nan values after merging

matches = matches.dropna(axis=1, how='all')
matches_primaryTitle = matches_primaryTitle.dropna(axis=1, how='all')

#cleaning

matches_primaryTitle.columns = matches_primaryTitle.columns.str.replace("_y", "")
matches_primaryTitle.columns = matches_primaryTitle.columns.str.replace("_x","")
matches_primaryTitle = matches_primaryTitle.reset_index(drop=True)
matches = matches.reset_index(drop=True)
matches_primaryTitle.drop(columns=['_merge'], inplace=True)
matches.drop(columns=['_merge'], inplace=True)
matches_primaryTitle.rename(columns={'year_diff_prim' : 'year_diff'}, inplace= True)

matches


,movie,movie_country,cites,cites_country,title,year,primaryTitle_citation,year_citation,title_citation_normalized,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,genres,originalTitle_normalized,primaryTitle_normalized,year_diff
0,#DevinAuditions (2016),Canada,Alien (1979),UK,#DevinAuditions,2016.0,Alien,1979.0,alien,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",alien,alien,0.0
1,#DevinAuditions (2016),Canada,Alien (1979),USA,#DevinAuditions,2016.0,Alien,1979.0,alien,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",alien,alien,0.0
2,.com for Murder (2002),USA,Alien (1979),UK,.com for Murder,2002.0,Alien,1979.0,alien,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",alien,alien,0.0
3,.com for Murder (2002),USA,Alien (1979),USA,.com for Murder,2002.0,Alien,1979.0,alien,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",alien,alien,0.0
4,30 Minutes or Less (2011),Germany,Alien (1979),UK,30 Minutes or Less,2011.0,Alien,1979.0,alien,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",alien,alien,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358636,è solo questione di punti di vista (2012),Italy,Letto a tre piazze (1960),Italy,è solo questione di punti di vista,2012.0,Letto a tre piazze,1960.0,lettoatrepiazze,tt0054023,movie,Letto a tre piazze,Letto a tre piazze,0,1960.0,Comedy,lettoatrepiazze,lettoatrepiazze,0.0
358637,è solo questione di tempo (2013),Italy,Il successo (1963),Italy,è solo questione di tempo,2013.0,Il successo,1963.0,ilsuccesso,tt0057540,movie,Il successo,Il successo,0,1963.0,Comedy,ilsuccesso,ilsuccesso,0.0
358638,è solo questione di tempo (2013),Italy,Il successo (1963),Italy,è solo questione di tempo,2013.0,Il successo,1963.0,ilsuccesso,tt1248945,tvMovie,Il successo,Il successo,0,1963.0,<NA>,ilsuccesso,ilsuccesso,0.0
358639,è solo questione di tempo (2013),Italy,Il successo (1963),France,è solo questione di tempo,2013.0,Il successo,1963.0,ilsuccesso,tt0057540,movie,Il successo,Il successo,0,1963.0,Comedy,ilsuccesso,ilsuccesso,0.0


In [19]:
#create final df with matches
final_citations = pd.concat([matches,matches_primaryTitle],ignore_index=True)


final_citations['match_primaryTitle'].replace({'both': 'True'},inplace=True)
final_citations['match_primaryTitle'] = final_citations['match_primaryTitle'].cat.add_categories(['False']).fillna('False')
final_citations.drop(columns=['originalTitle_normalized','primaryTitle_normalized','title_citation_normalized'],inplace=True)
final_citations


,movie,movie_country,cites,cites_country,title,year,primaryTitle_citation,year_citation,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,genres,year_diff,match_primaryTitle
0,#DevinAuditions (2016),Canada,Alien (1979),UK,#DevinAuditions,2016.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
1,#DevinAuditions (2016),Canada,Alien (1979),USA,#DevinAuditions,2016.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
2,.com for Murder (2002),USA,Alien (1979),UK,.com for Murder,2002.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
3,.com for Murder (2002),USA,Alien (1979),USA,.com for Murder,2002.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
4,30 Minutes or Less (2011),Germany,Alien (1979),UK,30 Minutes or Less,2011.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359436,Yin yang lu shi ba zhi Gui shang shen (2003),Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi ba zhi Gui shang shen,2003.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True
359437,Yin yang lu shi jiu zhi Wo dui yan jian dao ye...,Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi jiu zhi Wo dui yan jian dao ye,2003.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True
359438,Yin yang lu shi liu zhi hui dao wu xia shi dai...,Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi liu zhi hui dao wu xia shi dai,2002.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True
359439,Yin yang lu shi qi zhi jian fang you gui (2002),Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi qi zhi jian fang you gui,2002.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True


In [20]:
# final_citations[final_citations.duplicated()]

final_citations.drop_duplicates(inplace=True)
final_citations.columns = final_citations.columns.str.replace("_citation", "_cit")

final_citations= final_citations.add_suffix('_cit')
final_citations.rename(columns={'movie_cit':'movie', 'movie_country_cit':'movie_country','title_cit':'title','year_cit':'year'} ,inplace=True)
final_citations.columns = final_citations.columns.str.replace("cites_", "")
final_citations.columns = final_citations.columns.str.replace("_cit_cit", "_cit")

final_citations


,movie,movie_country,cit,country_cit,title,year,primaryTitle_cit,year_cit,tconst_cit,titleType_cit,primaryTitle_cit,originalTitle_cit,isAdult_cit,startYear_cit,genres_cit,year_diff_cit,match_primaryTitle_cit
0,#DevinAuditions (2016),Canada,Alien (1979),UK,#DevinAuditions,2016.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
1,#DevinAuditions (2016),Canada,Alien (1979),USA,#DevinAuditions,2016.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
2,.com for Murder (2002),USA,Alien (1979),UK,.com for Murder,2002.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
3,.com for Murder (2002),USA,Alien (1979),USA,.com for Murder,2002.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
4,30 Minutes or Less (2011),Germany,Alien (1979),UK,30 Minutes or Less,2011.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359436,Yin yang lu shi ba zhi Gui shang shen (2003),Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi ba zhi Gui shang shen,2003.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True
359437,Yin yang lu shi jiu zhi Wo dui yan jian dao ye...,Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi jiu zhi Wo dui yan jian dao ye,2003.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True
359438,Yin yang lu shi liu zhi hui dao wu xia shi dai...,Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi liu zhi hui dao wu xia shi dai,2002.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True
359439,Yin yang lu shi qi zhi jian fang you gui (2002),Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi qi zhi jian fang you gui,2002.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True


In [21]:

final_citations


,movie,movie_country,cit,country_cit,title,year,primaryTitle_cit,year_cit,tconst_cit,titleType_cit,primaryTitle_cit,originalTitle_cit,isAdult_cit,startYear_cit,genres_cit,year_diff_cit,match_primaryTitle_cit
0,#DevinAuditions (2016),Canada,Alien (1979),UK,#DevinAuditions,2016.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
1,#DevinAuditions (2016),Canada,Alien (1979),USA,#DevinAuditions,2016.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
2,.com for Murder (2002),USA,Alien (1979),UK,.com for Murder,2002.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
3,.com for Murder (2002),USA,Alien (1979),USA,.com for Murder,2002.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
4,30 Minutes or Less (2011),Germany,Alien (1979),UK,30 Minutes or Less,2011.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359436,Yin yang lu shi ba zhi Gui shang shen (2003),Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi ba zhi Gui shang shen,2003.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True
359437,Yin yang lu shi jiu zhi Wo dui yan jian dao ye...,Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi jiu zhi Wo dui yan jian dao ye,2003.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True
359438,Yin yang lu shi liu zhi hui dao wu xia shi dai...,Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi liu zhi hui dao wu xia shi dai,2002.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True
359439,Yin yang lu shi qi zhi jian fang you gui (2002),Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi qi zhi jian fang you gui,2002.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True


In [22]:
#MERGE OF MOVIES AND CITATIONS
#normalize movie names

movies_cit_countries['movie_normalized'] = movies_cit_countries['title'].apply(normalize_string)
movies_cit_countries[movies_cit_countries.duplicated()]

,movie,movie_country,cites,cites_country,title,year,primaryTitle_citation,year_citation,title_citation_normalized,movie_normalized
322,(T)Raumschiff Surprise - Periode 1 (2004),Germany,2001: A Space Odyssey (1968),UK,(T)Raumschiff Surprise - Periode 1,2004,2001: A Space Odyssey,1968,2001aspaceodyssey,traumschiffsurpriseperiode1
323,(T)Raumschiff Surprise - Periode 1 (2004),Germany,2001: A Space Odyssey (1968),USA,(T)Raumschiff Surprise - Periode 1,2004,2001: A Space Odyssey,1968,2001aspaceodyssey,traumschiffsurpriseperiode1
325,(T)Raumschiff Surprise - Periode 1 (2004),Germany,Back to the Future Part III (1990),USA,(T)Raumschiff Surprise - Periode 1,2004,Back to the Future Part III,1990,backtothefuturepartiii,traumschiffsurpriseperiode1
327,(T)Raumschiff Surprise - Periode 1 (2004),Germany,Der Schuh des Manitu (2001),Germany,(T)Raumschiff Surprise - Periode 1,2004,Der Schuh des Manitu,2001,derschuhdesmanitu,traumschiffsurpriseperiode1
374,...Promises to Keep (1974),USA,The Yakuza (1974),USA,...Promises to Keep,1974,The Yakuza,1974,theyakuza,promisestokeep
...,...,...,...,...,...,...,...,...,...,...
332320,xXx: Return of Xander Cage (2017),China,xXx: State of the Union (2005),USA,xXx: Return of Xander Cage,2017,xXx: State of the Union,2005,xxxstateoftheunion,xxxreturnofxandercage
332321,xXx: Return of Xander Cage (2017),Canada,xXx: State of the Union (2005),USA,xXx: Return of Xander Cage,2017,xXx: State of the Union,2005,xxxstateoftheunion,xxxreturnofxandercage
332322,xXx: Return of Xander Cage (2017),USA,xXx: State of the Union (2005),USA,xXx: Return of Xander Cage,2017,xXx: State of the Union,2005,xxxstateoftheunion,xxxreturnofxandercage
332344,¡Maldito bastardo! (2008),Spain,La consulta del Dr. Natalio (2004),Spain,¡Maldito bastardo!,2008,La consulta del Dr. Natalio,2004,laconsultadeldrnatalio,malditobastardo


In [23]:
#merge 
movies_df = pd.merge(movies_cit_countries, title_name_table,
             left_on='movie_normalized',
             right_on='originalTitle_normalized', how='outer', indicator=True)

movies_df

,movie,movie_country,cites,cites_country,title,year,primaryTitle_citation,year_citation,title_citation_normalized,movie_normalized,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,genres,originalTitle_normalized,primaryTitle_normalized,_merge
0,#DevinAuditions (2016),Canada,Alien (1979),UK,#DevinAuditions,2016.0,Alien,1979.0,alien,devinauditions,tt6231178,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",devinauditions,devinauditions,both
1,#DevinAuditions (2016),Canada,Alien (1979),USA,#DevinAuditions,2016.0,Alien,1979.0,alien,devinauditions,tt6231178,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",devinauditions,devinauditions,both
2,#DevinAuditions (2016),Canada,Die Hard (1988),USA,#DevinAuditions,2016.0,Die Hard,1988.0,diehard,devinauditions,tt6231178,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",devinauditions,devinauditions,both
3,#DevinAuditions (2016),Canada,Ferris Bueller's Day Off (1986),USA,#DevinAuditions,2016.0,Ferris Bueller's Day Off,1986.0,ferrisbuellersdayoff,devinauditions,tt6231178,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",devinauditions,devinauditions,both
4,#DevinAuditions (2016),Canada,Fight Club (1999),USA,#DevinAuditions,2016.0,Fight Club,1999.0,fightclub,devinauditions,tt6231178,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",devinauditions,devinauditions,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2457536,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,tt9916706,movie,Dankyavar Danka,Dankyavar Danka,0,2013.0,Comedy,dankyavardanka,dankyavardanka,right_only
2457537,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,tt9916724,short,Hay Que Ser Paciente,Hay Que Ser Paciente,0,2015.0,"Documentary,Short",hayqueserpaciente,hayqueserpaciente,right_only
2457538,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,tt9916730,movie,6 Gunn,6 Gunn,0,2017.0,<NA>,6gunn,6gunn,right_only
2457539,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,tt9916754,movie,Chico Albuquerque - Revelações,Chico Albuquerque - Revelações,0,2013.0,Documentary,chicoalbuquerquerevelacoes,chicoalbuquerquerevelacoes,right_only


In [24]:
matches_movies = movies_df[movies_df['_merge'] == "both"].copy()
matches_movies['year_diff'] = matches_movies['year'] - matches_movies['startYear']
matches_movies = matches_movies.loc[matches_movies['year_diff'].between(-3, 3)]
matches_movies

,movie,movie_country,cites,cites_country,title,year,primaryTitle_citation,year_citation,title_citation_normalized,movie_normalized,...,titleType,primaryTitle,originalTitle,isAdult,startYear,genres,originalTitle_normalized,primaryTitle_normalized,_merge,year_diff
0,#DevinAuditions (2016),Canada,Alien (1979),UK,#DevinAuditions,2016.0,Alien,1979.0,alien,devinauditions,...,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",devinauditions,devinauditions,both,0.0
1,#DevinAuditions (2016),Canada,Alien (1979),USA,#DevinAuditions,2016.0,Alien,1979.0,alien,devinauditions,...,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",devinauditions,devinauditions,both,0.0
2,#DevinAuditions (2016),Canada,Die Hard (1988),USA,#DevinAuditions,2016.0,Die Hard,1988.0,diehard,devinauditions,...,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",devinauditions,devinauditions,both,0.0
3,#DevinAuditions (2016),Canada,Ferris Bueller's Day Off (1986),USA,#DevinAuditions,2016.0,Ferris Bueller's Day Off,1986.0,ferrisbuellersdayoff,devinauditions,...,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",devinauditions,devinauditions,both,0.0
4,#DevinAuditions (2016),Canada,Fight Club (1999),USA,#DevinAuditions,2016.0,Fight Club,1999.0,fightclub,devinauditions,...,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",devinauditions,devinauditions,both,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882066,è solo questione di punti di vista (2012),Italy,"Un genio, due compari, un pollo (1975)",Italy,è solo questione di punti di vista,2012.0,"Un genio, due compari, un pollo",1975.0,ungenioduecompariunpollo,esoloquestionedipuntidivista,...,movie,è solo questione di punti di vista,è solo questione di punti di vista,0,2012.0,"Action,Adventure,Comedy",esoloquestionedipuntidivista,esoloquestionedipuntidivista,both,0.0
882067,è solo questione di punti di vista (2012),Italy,"Un genio, due compari, un pollo (1975)",France,è solo questione di punti di vista,2012.0,"Un genio, due compari, un pollo",1975.0,ungenioduecompariunpollo,esoloquestionedipuntidivista,...,movie,è solo questione di punti di vista,è solo questione di punti di vista,0,2012.0,"Action,Adventure,Comedy",esoloquestionedipuntidivista,esoloquestionedipuntidivista,both,0.0
882068,è solo questione di punti di vista (2012),Italy,"Un genio, due compari, un pollo (1975)",West Germany,è solo questione di punti di vista,2012.0,"Un genio, due compari, un pollo",1975.0,ungenioduecompariunpollo,esoloquestionedipuntidivista,...,movie,è solo questione di punti di vista,è solo questione di punti di vista,0,2012.0,"Action,Adventure,Comedy",esoloquestionedipuntidivista,esoloquestionedipuntidivista,both,0.0
882069,è solo questione di tempo (2013),Italy,Il successo (1963),Italy,è solo questione di tempo,2013.0,Il successo,1963.0,ilsuccesso,esoloquestioneditempo,...,movie,è solo questione di tempo,è solo questione di tempo,0,2013.0,Comedy,esoloquestioneditempo,esoloquestioneditempo,both,0.0


In [25]:
imdb_unmatched_1 = movies_df[movies_df['_merge'] == "right_only"].copy()
links_unmatched_1 = movies_df[movies_df['_merge'] == "left_only"].copy()
imdb_unmatched_1

movies_candidate_primary = pd.merge(links_unmatched_1, imdb_unmatched_1,
             left_on=['movie_normalized'],
             right_on=['primaryTitle_normalized'],how='outer', indicator="match_primaryTitle")


movies_match_primary = movies_candidate_primary[movies_candidate_primary['match_primaryTitle'] == "both"].copy()
movies_match_primary['year_diff_prim'] = movies_match_primary['year_x'] - movies_match_primary['startYear_y']
movies_match_primary = movies_match_primary.loc[movies_match_primary['year_diff_prim'].between(-3, 3)]
movies_match_primary

,movie_x,movie_country_x,cites_x,cites_country_x,title_x,year_x,primaryTitle_citation_x,year_citation_x,title_citation_normalized_x,movie_normalized_x,...,primaryTitle_y,originalTitle_y,isAdult_y,startYear_y,genres_y,originalTitle_normalized_y,primaryTitle_normalized_y,_merge_y,match_primaryTitle,year_diff_prim
135,Angels Wash Their Faces (1939),USA,Code of the Streets (1939),USA,Angels Wash Their Faces,1939.0,Code of the Streets,1939.0,codeofthestreets,angelswashtheirfaces,...,Angels Wash Their Faces,The Angels Wash Their Faces,0,1939.0,"Drama,Romance",theangelswashtheirfaces,angelswashtheirfaces,right_only,both,0.0
136,Angels Wash Their Faces (1939),USA,Hell's Kitchen (1939),USA,Angels Wash Their Faces,1939.0,Hell's Kitchen,1939.0,hellskitchen,angelswashtheirfaces,...,Angels Wash Their Faces,The Angels Wash Their Faces,0,1939.0,"Drama,Romance",theangelswashtheirfaces,angelswashtheirfaces,right_only,both,0.0
137,Angels Wash Their Faces (1939),USA,They Made Me a Criminal (1939),USA,Angels Wash Their Faces,1939.0,They Made Me a Criminal,1939.0,theymademeacriminal,angelswashtheirfaces,...,Angels Wash Their Faces,The Angels Wash Their Faces,0,1939.0,"Drama,Romance",theangelswashtheirfaces,angelswashtheirfaces,right_only,both,0.0
138,Angels Wash Their Faces (1939),USA,Angels with Dirty Faces (1938),USA,Angels Wash Their Faces,1939.0,Angels with Dirty Faces,1938.0,angelswithdirtyfaces,angelswashtheirfaces,...,Angels Wash Their Faces,The Angels Wash Their Faces,0,1939.0,"Drama,Romance",theangelswashtheirfaces,angelswashtheirfaces,right_only,both,0.0
322,Blood for Dracula (1974),Italy,Flesh for Frankenstein (1973),USA,Blood for Dracula,1974.0,Flesh for Frankenstein,1973.0,fleshforfrankenstein,bloodfordracula,...,Blood for Dracula,Sangue per Dracula,0,1974.0,Horror,sangueperdracula,bloodfordracula,right_only,both,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4696,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),Canada,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,xmenthelaststand,xmendarkphoenix,...,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",darkphoenix,xmendarkphoenix,right_only,both,-1.0
4697,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),USA,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,xmenthelaststand,xmendarkphoenix,...,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",darkphoenix,xmendarkphoenix,right_only,both,-1.0
4698,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),UK,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,xmenthelaststand,xmendarkphoenix,...,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",darkphoenix,xmendarkphoenix,right_only,both,-1.0
4699,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),USA,X-Men: Dark Phoenix,2018.0,X: First Class,2011.0,xfirstclass,xmendarkphoenix,...,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",darkphoenix,xmendarkphoenix,right_only,both,-1.0


In [26]:
movies_candidate_primary[movies_candidate_primary['match_primaryTitle']=='left_only']
#replace Zui jia pai dang 3: Nv huang mi ling with "nu huang mi..."

,movie_x,movie_country_x,cites_x,cites_country_x,title_x,year_x,primaryTitle_citation_x,year_citation_x,title_citation_normalized_x,movie_normalized_x,...,titleType_y,primaryTitle_y,originalTitle_y,isAdult_y,startYear_y,genres_y,originalTitle_normalized_y,primaryTitle_normalized_y,_merge_y,match_primaryTitle
0,...Be yom hashlishi (2010),Israel,Ai no korîda (1976),Japan,...Be yom hashlishi,2010.0,Ai no korîda,1976.0,ainokorida,beyomhashlishi,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
1,...Be yom hashlishi (2010),Israel,Ai no korîda (1976),France,...Be yom hashlishi,2010.0,Ai no korîda,1976.0,ainokorida,beyomhashlishi,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
2,100 Tears (2007),USA,Unearthed (2004),USA,100 Tears,2007.0,Unearthed,2004.0,unearthed,100tears,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
3,100 lat w kinie (1995),Poland,Amator (1979),Poland,100 lat w kinie,1995.0,Amator,1979.0,amator,100latwkinie,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
4,100 lat w kinie (1995),UK,Amator (1979),Poland,100 lat w kinie,1995.0,Amator,1979.0,amator,100latwkinie,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4963,Zui jia pai dang (1982),Hong Kong,Goldfinger (1964),UK,Zui jia pai dang,1982.0,Goldfinger,1964.0,goldfinger,zuijiapaidang,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
4964,Zui jia pai dang (1982),Hong Kong,The Godfather (1972),USA,Zui jia pai dang,1982.0,The Godfather,1972.0,thegodfather,zuijiapaidang,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
4965,Zui jia pai dang (1982),Hong Kong,The Pink Panther (1963),USA,Zui jia pai dang,1982.0,The Pink Panther,1963.0,thepinkpanther,zuijiapaidang,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
4966,Zui jia pai dang: Zui jie pai dang (1997),Hong Kong,Zui jia pai dang (1982),Hong Kong,Zui jia pai dang: Zui jie pai dang,1997.0,Zui jia pai dang,1982.0,zuijiapaidang,zuijiapaidangzuijiepaidang,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only


In [27]:
#check missing values
matches_movies = matches_movies.dropna(axis=1, how='all')
movies_match_primary = movies_match_primary.dropna(axis=1, how='all')

movies_match_primary.columns = movies_match_primary.columns.str.replace("_y", "")
movies_match_primary.columns = movies_match_primary.columns.str.replace("_x","")

matches_movies = matches_movies.reset_index(drop=True)
movies_match_primary = movies_match_primary.reset_index(drop=True)
# matches_movies.drop(columns=['_merge'], inplace=True)
movies_match_primary.drop(columns=['_merge'], inplace=True)
movies_match_primary.rename(columns={'year_diff_prim' : 'year_diff'}, inplace= True)

movies_match_primary


,movie,movie_country,cites,cites_country,title,year,primaryTitle_citation,year_citation,title_citation_normalized,movie_normalized,...,titleType,primaryTitle,originalTitle,isAdult,startYear,genres,originalTitle_normalized,primaryTitle_normalized,match_primaryTitle,year_diff
0,Angels Wash Their Faces (1939),USA,Code of the Streets (1939),USA,Angels Wash Their Faces,1939.0,Code of the Streets,1939.0,codeofthestreets,angelswashtheirfaces,...,movie,Angels Wash Their Faces,The Angels Wash Their Faces,0,1939.0,"Drama,Romance",theangelswashtheirfaces,angelswashtheirfaces,both,0.0
1,Angels Wash Their Faces (1939),USA,Hell's Kitchen (1939),USA,Angels Wash Their Faces,1939.0,Hell's Kitchen,1939.0,hellskitchen,angelswashtheirfaces,...,movie,Angels Wash Their Faces,The Angels Wash Their Faces,0,1939.0,"Drama,Romance",theangelswashtheirfaces,angelswashtheirfaces,both,0.0
2,Angels Wash Their Faces (1939),USA,They Made Me a Criminal (1939),USA,Angels Wash Their Faces,1939.0,They Made Me a Criminal,1939.0,theymademeacriminal,angelswashtheirfaces,...,movie,Angels Wash Their Faces,The Angels Wash Their Faces,0,1939.0,"Drama,Romance",theangelswashtheirfaces,angelswashtheirfaces,both,0.0
3,Angels Wash Their Faces (1939),USA,Angels with Dirty Faces (1938),USA,Angels Wash Their Faces,1939.0,Angels with Dirty Faces,1938.0,angelswithdirtyfaces,angelswashtheirfaces,...,movie,Angels Wash Their Faces,The Angels Wash Their Faces,0,1939.0,"Drama,Romance",theangelswashtheirfaces,angelswashtheirfaces,both,0.0
4,Blood for Dracula (1974),Italy,Flesh for Frankenstein (1973),USA,Blood for Dracula,1974.0,Flesh for Frankenstein,1973.0,fleshforfrankenstein,bloodfordracula,...,movie,Blood for Dracula,Sangue per Dracula,0,1974.0,Horror,sangueperdracula,bloodfordracula,both,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),Canada,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,xmenthelaststand,xmendarkphoenix,...,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",darkphoenix,xmendarkphoenix,both,-1.0
401,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),USA,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,xmenthelaststand,xmendarkphoenix,...,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",darkphoenix,xmendarkphoenix,both,-1.0
402,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),UK,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,xmenthelaststand,xmendarkphoenix,...,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",darkphoenix,xmendarkphoenix,both,-1.0
403,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),USA,X-Men: Dark Phoenix,2018.0,X: First Class,2011.0,xfirstclass,xmendarkphoenix,...,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",darkphoenix,xmendarkphoenix,both,-1.0


In [28]:
final_movies = pd.concat([matches_movies,movies_match_primary],ignore_index=True)
final_movies['match_primaryTitle'].replace({'both': 'True'},inplace=True)
final_movies['match_primaryTitle'] = final_movies['match_primaryTitle'].cat.add_categories(['False']).fillna('False')
final_movies.columns = final_movies.columns.str.replace("_citation", "_cit")
final_movies.drop(columns=['title_cit_normalized','movie_normalized','originalTitle_normalized','primaryTitle_normalized','_merge'], inplace=True)
final_movies

,movie,movie_country,cites,cites_country,title,year,primaryTitle_cit,year_cit,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,genres,year_diff,match_primaryTitle
0,#DevinAuditions (2016),Canada,Alien (1979),UK,#DevinAuditions,2016.0,Alien,1979.0,tt6231178,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",0.0,False
1,#DevinAuditions (2016),Canada,Alien (1979),USA,#DevinAuditions,2016.0,Alien,1979.0,tt6231178,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",0.0,False
2,#DevinAuditions (2016),Canada,Die Hard (1988),USA,#DevinAuditions,2016.0,Die Hard,1988.0,tt6231178,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",0.0,False
3,#DevinAuditions (2016),Canada,Ferris Bueller's Day Off (1986),USA,#DevinAuditions,2016.0,Ferris Bueller's Day Off,1986.0,tt6231178,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",0.0,False
4,#DevinAuditions (2016),Canada,Fight Club (1999),USA,#DevinAuditions,2016.0,Fight Club,1999.0,tt6231178,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372573,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),Canada,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,tt6565702,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",-1.0,True
372574,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),USA,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,tt6565702,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",-1.0,True
372575,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),UK,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,tt6565702,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",-1.0,True
372576,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),USA,X-Men: Dark Phoenix,2018.0,X: First Class,2011.0,tt6565702,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",-1.0,True


In [29]:
final_movies

,movie,movie_country,cites,cites_country,title,year,primaryTitle_cit,year_cit,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,genres,year_diff,match_primaryTitle
0,#DevinAuditions (2016),Canada,Alien (1979),UK,#DevinAuditions,2016.0,Alien,1979.0,tt6231178,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",0.0,False
1,#DevinAuditions (2016),Canada,Alien (1979),USA,#DevinAuditions,2016.0,Alien,1979.0,tt6231178,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",0.0,False
2,#DevinAuditions (2016),Canada,Die Hard (1988),USA,#DevinAuditions,2016.0,Die Hard,1988.0,tt6231178,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",0.0,False
3,#DevinAuditions (2016),Canada,Ferris Bueller's Day Off (1986),USA,#DevinAuditions,2016.0,Ferris Bueller's Day Off,1986.0,tt6231178,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",0.0,False
4,#DevinAuditions (2016),Canada,Fight Club (1999),USA,#DevinAuditions,2016.0,Fight Club,1999.0,tt6231178,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372573,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),Canada,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,tt6565702,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",-1.0,True
372574,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),USA,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,tt6565702,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",-1.0,True
372575,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),UK,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,tt6565702,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",-1.0,True
372576,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),USA,X-Men: Dark Phoenix,2018.0,X: First Class,2011.0,tt6565702,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",-1.0,True


In [30]:
final_movies = final_movies.drop_duplicates()
final_citations = final_citations.drop_duplicates()
final_citations

,movie,movie_country,cit,country_cit,title,year,primaryTitle_cit,year_cit,tconst_cit,titleType_cit,primaryTitle_cit,originalTitle_cit,isAdult_cit,startYear_cit,genres_cit,year_diff_cit,match_primaryTitle_cit
0,#DevinAuditions (2016),Canada,Alien (1979),UK,#DevinAuditions,2016.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
1,#DevinAuditions (2016),Canada,Alien (1979),USA,#DevinAuditions,2016.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
2,.com for Murder (2002),USA,Alien (1979),UK,.com for Murder,2002.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
3,.com for Murder (2002),USA,Alien (1979),USA,.com for Murder,2002.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
4,30 Minutes or Less (2011),Germany,Alien (1979),UK,30 Minutes or Less,2011.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359436,Yin yang lu shi ba zhi Gui shang shen (2003),Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi ba zhi Gui shang shen,2003.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True
359437,Yin yang lu shi jiu zhi Wo dui yan jian dao ye...,Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi jiu zhi Wo dui yan jian dao ye,2003.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True
359438,Yin yang lu shi liu zhi hui dao wu xia shi dai...,Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi liu zhi hui dao wu xia shi dai,2002.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True
359439,Yin yang lu shi qi zhi jian fang you gui (2002),Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi qi zhi jian fang you gui,2002.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True


In [36]:
final_movies = final_movies.dropna(axis=1, how='all')
final_movies

final_citations = final_citations.dropna(axis=1, how='all')
final_citations

final_citations['titleType_cit'].unique()
final_movies

,movie,movie_country,cites,cites_country,title,year,primaryTitle_cit,year_cit,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,genres,year_diff,match_primaryTitle
0,#DevinAuditions (2016),Canada,Alien (1979),UK,#DevinAuditions,2016.0,Alien,1979.0,tt6231178,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",0.0,False
1,#DevinAuditions (2016),Canada,Alien (1979),USA,#DevinAuditions,2016.0,Alien,1979.0,tt6231178,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",0.0,False
2,#DevinAuditions (2016),Canada,Die Hard (1988),USA,#DevinAuditions,2016.0,Die Hard,1988.0,tt6231178,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",0.0,False
3,#DevinAuditions (2016),Canada,Ferris Bueller's Day Off (1986),USA,#DevinAuditions,2016.0,Ferris Bueller's Day Off,1986.0,tt6231178,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",0.0,False
4,#DevinAuditions (2016),Canada,Fight Club (1999),USA,#DevinAuditions,2016.0,Fight Club,1999.0,tt6231178,short,#DevinAuditions,#DevinAuditions,0,2016.0,"Comedy,Short",0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372573,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),Canada,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,tt6565702,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",-1.0,True
372574,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),USA,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,tt6565702,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",-1.0,True
372575,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),UK,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,tt6565702,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",-1.0,True
372576,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),USA,X-Men: Dark Phoenix,2018.0,X: First Class,2011.0,tt6565702,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",-1.0,True


In [60]:
citations_data = final_citations[['movie', 'movie_country','cit','country_cit','primaryTitle_cit','year_cit','tconst_cit','titleType_cit','genres_cit']]

citations_data = citations_data.loc[:, ~citations_data.columns.duplicated()]
citations_data


movies_data  = final_movies[['movie', 'movie_country','cites','cites_country','title','year','tconst','titleType','genres']]

movies_data



,movie,movie_country,cit,country_cit,primaryTitle_cit,year_cit,tconst_cit,titleType_cit,genres_cit
0,#DevinAuditions (2016),Canada,Alien (1979),UK,Alien,1979.0,tt0078748,movie,"Horror,Sci-Fi"
1,#DevinAuditions (2016),Canada,Alien (1979),USA,Alien,1979.0,tt0078748,movie,"Horror,Sci-Fi"
2,.com for Murder (2002),USA,Alien (1979),UK,Alien,1979.0,tt0078748,movie,"Horror,Sci-Fi"
3,.com for Murder (2002),USA,Alien (1979),USA,Alien,1979.0,tt0078748,movie,"Horror,Sci-Fi"
4,30 Minutes or Less (2011),Germany,Alien (1979),UK,Alien,1979.0,tt0078748,movie,"Horror,Sci-Fi"
...,...,...,...,...,...,...,...,...,...
359436,Yin yang lu shi ba zhi Gui shang shen (2003),Hong Kong,Troublesome Night 11 (2001),Hong Kong,Troublesome Night 11,2001.0,tt0297441,movie,"Comedy,Horror"
359437,Yin yang lu shi jiu zhi Wo dui yan jian dao ye...,Hong Kong,Troublesome Night 11 (2001),Hong Kong,Troublesome Night 11,2001.0,tt0297441,movie,"Comedy,Horror"
359438,Yin yang lu shi liu zhi hui dao wu xia shi dai...,Hong Kong,Troublesome Night 11 (2001),Hong Kong,Troublesome Night 11,2001.0,tt0297441,movie,"Comedy,Horror"
359439,Yin yang lu shi qi zhi jian fang you gui (2002),Hong Kong,Troublesome Night 11 (2001),Hong Kong,Troublesome Night 11,2001.0,tt0297441,movie,"Comedy,Horror"


In [ ]:
graph_df.info()

In [63]:

graph_df = pd.merge(movies_data, citations_data, left_on=['movie','movie_country','cites','cites_country'],
                    right_on = ['movie','movie_country','cit','country_cit'])

graph_df

,movie,movie_country,cites,cites_country,title,year,tconst,titleType,genres,cit,country_cit,primaryTitle_cit,year_cit,tconst_cit,titleType_cit,genres_cit
0,#DevinAuditions (2016),Canada,Alien (1979),UK,#DevinAuditions,2016.0,tt6231178,short,"Comedy,Short",Alien (1979),UK,Alien,1979.0,tt0078748,movie,"Horror,Sci-Fi"
1,#DevinAuditions (2016),Canada,Alien (1979),USA,#DevinAuditions,2016.0,tt6231178,short,"Comedy,Short",Alien (1979),USA,Alien,1979.0,tt0078748,movie,"Horror,Sci-Fi"
2,#DevinAuditions (2016),Canada,Die Hard (1988),USA,#DevinAuditions,2016.0,tt6231178,short,"Comedy,Short",Die Hard (1988),USA,Die Hard,1988.0,tt0095016,movie,"Action,Thriller"
3,#DevinAuditions (2016),Canada,Ferris Bueller's Day Off (1986),USA,#DevinAuditions,2016.0,tt6231178,short,"Comedy,Short",Ferris Bueller's Day Off (1986),USA,Ferris Bueller's Day Off,1986.0,tt0091042,movie,Comedy
4,#DevinAuditions (2016),Canada,Fight Club (1999),USA,#DevinAuditions,2016.0,tt6231178,short,"Comedy,Short",Fight Club (1999),USA,Fight Club,1999.0,tt0137523,movie,Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403277,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),Canada,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X-Men: The Last Stand (2006),Canada,X-Men: The Last Stand,2006.0,tt0376994,movie,"Action,Adventure,Sci-Fi"
403278,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),USA,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X-Men: The Last Stand (2006),USA,X-Men: The Last Stand,2006.0,tt0376994,movie,"Action,Adventure,Sci-Fi"
403279,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),UK,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X-Men: The Last Stand (2006),UK,X-Men: The Last Stand,2006.0,tt0376994,movie,"Action,Adventure,Sci-Fi"
403280,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),USA,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),USA,X: First Class,2011.0,tt1270798,movie,"Action,Sci-Fi"


In [64]:
graph_df['movie'].unique()

<StringArray>
[                                     '#DevinAuditions (2016)',
                                        '#FromJennifer (2017)',
                                                 '#Rip (2013)',
                                               'R.I.P. (2001)',
                                               'R.I.P. (2013)',
                                         '#TubeClash02 (2016)',
 '#chicagoGirl: The Social Network Takes on a Dictator (2013)',
                                        '#twitterkills (2014)',
                                                '#will (2016)',
                                            '$10 Raise (1935)',
 ...
                                   'Tea with Mussolini (1999)',
                    "The Crime Doctor's Strangest Case (1943)",
                         'The Killer Bean 2: The Party (2000)',
                            'The Prince and the Dybbuk (2017)',
                             'The Young Black Stallion (2003)',
                     

In [65]:
graph_df.to_csv('graph_1_movies_cit.csv')

In [ ]:
#45579
#44517 --> full merge
# 44517 --> merge on all 
